## In this lab we are going to implement the following algorithms from the course:
 - policy evaluation
 - policy improvement
 - policy iteration
 - value iteration
 
You can review these either from the course, or read Chapter 4 from the book Introduction to Reinforcement Learning by Sutton (attached to Files section in Teams)
 
We'll work on the Frozen Lake environment: https://www.gymlibrary.dev/environments/toy_text/frozen_lake/

Read the description, but look at their github implementation of the environment later, after implementing this lab.
For now, follow the code in the cell below, function `runEpisode' to see how we load and interact with this environment
Remember that all environments from gym have a similar structure and it is important to understand the API !

### Note that in order to solve our problems using Bellman equations (DP method) as we do in this laboratory means two things:
 - We have full access to the model dynamics - which we do, as you see below with matrix P. 
   In the continuation of the course, as in most of the problems this information is not available because it is very difficult to obtain !
 - We can represent numerically all the states and actions. Is this feasible for a self driving car ?
 - However, when the above requirements can be satisfied, NOTE that the solutions are OPTIMAL !. Choose your algorithms wisely !

In [1]:
# First ensure that we can install numpy and gym here then import them
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install gym

import numpy as np
import gym
import time
np.set_printoptions(precision=3)

### This is how we load the environment. We give it the name and the parameters (which are sent to the __init__ function of the respective class)
### Try to switch the name of the map between 4x4 and 8x8 to see performance.
### is_splippery = False means the environment is deterministic, while True means that doing an action "a" in state "s" can cause movement to different states "s'" ! (non-deterministic !)

In [2]:
env = gym.make("FrozenLake-v1",  map_name="4x4", is_slippery=False)

"""
You can see from documentation that this environment contains three main things inside:
	P: nested dictionary 
	    (simulates the  p(s',r | s, a) = the probability of being in state s, applying action a and landing in state s' with a reward of r)
		From gym.core.Environment:
		For each pair of states in [1, nS] and actions in [1, nA], P[state][action] is a
		tuple of the form (probability, nextstate, reward, terminal) where
			- probability: float
				the probability of transitioning from "state" to "nextstate" with "action"
			- nextstate: int
				denotes the state we transition to (in range [0, nS - 1])
			- reward: int
				either 0 or 1, the reward for transitioning from "state" to
				"nextstate" with "action"
			- terminal: bool
			  True when "nextstate" is a terminal state (hole or goal), False otherwise
	nS: int
		number of states in the environment
	nA: int
		number of actions in the environment
		Inside, they implement it with an enum:
		LEFT = 0
        DOWN = 1
        RIGHT = 2
        UP = 3
"""

def runEpisode(env, policy, maxSteps=100):
    """
    Parameters
    ----------
    env: gym.core.Environment
      Environment to play on. Must have nS, nA, and P as
      attributes.
    Policy: np.array of shape [env.nS]
      The action to take at a given state
    """
    
    # We count here the total
    total_reward = 0
    
    # THis is how we reset the environment to an initial state, it returns the observation.
    # As documented, in this case the observation is the state where the agent currently is positionaed, 
    #, which is a number in [0, nS-1]. We can use local function stateToRC to get the row and column of the agent
    # The action give is in range [0, nA-1], check the enum defined above to understand what each number means
    obs = env.reset() 
    for t in range(maxSteps):
        # Draw the environment on screen
        env.render() 
        # Sleep a bit between decisions
        time.sleep(0.25)
        
        # Here we sample an action from our policy, we consider it deterministically at this point
        action = policy[obs]
        
        # Hwere we interact with the enviornment. We give it an action to do and it returns back:
        # - the new observation (observable state by the agent),
        # - the reward of the action just made
        # - if the simulation is done (terminal state)
        # - last parameters is an "info" output, we are not interested in this one that's why we ignore the parameter
        newObs, reward, done, _ = env.step(action)
        print(f"Agent was in state {obs}, took action {action}, now in state {newObs}")
        obs = newObs
        
        total_reward += reward
        # Close the loop before maxSteps  if we are in a terminal state
        if done:
            break
   
    if not done:   
        print(f"The agent didn't reach a terminal state in {maxSteps} steps.")
    else:
        print(f"Episode reward: {total_reward}")
    env.render() # One last  rendering of the episode.

In [3]:
# Uncomment the code below to  create a random policy and see an episode in action.
# Run it several times and see your agent in action with a random deterministic policy :)
env.nA = 4
env.nS = 16
random_policy = np.random.choice(env.nA, size=env.nS)
print(random_policy)
runEpisode(env, random_policy, 10)


[2 0 3 3 2 2 1 1 2 0 0 1 2 3 0 0]
Agent was in state 0, took action 2, now in state 1
Agent was in state 1, took action 0, now in state 0
Agent was in state 0, took action 2, now in state 1
Agent was in state 1, took action 0, now in state 0
Agent was in state 0, took action 2, now in state 1
Agent was in state 1, took action 0, now in state 0
Agent was in state 0, took action 2, now in state 1
Agent was in state 1, took action 0, now in state 0
Agent was in state 0, took action 2, now in state 1
Agent was in state 1, took action 0, now in state 0
The agent didn't reach a terminal state in 10 steps.


In [4]:
### Now let's run the episode with this policy found by value iteration
### Note that you may need to increase max number of Steps or run it several times if you select is_slippery !
### Try to play with map namees and stochastic parameter is_slippery than run again all above cells including this

In [5]:
def policy_evaluation(P, nS, nA, policy, gamma=0.9, tol=1e-3):
    """Evaluate the value function from a given policy.
    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    policy: np.array[nS]
        The policy to evaluate. Maps states to actions, deterministic !
    tol: float
        Terminate policy evaluation when
            max |value_function(s) - prev_value_function(s)| < tol
    Returns
    -------
    value_function: np.ndarray[nS]
        The value function of the given policy, where value_function[s] is
        the value of state s
    """
    # Init with 0 for all states, 
    # Remember that terminal states MUST have 0 always whatever you initialize them with here
    value_function = np.zeros(nS) 

    ############################
    # YOUR IMPLEMENTATION HERE #
    maxChange = np.inf
    numIters = 0
    while maxChange > tol:
        numIters += 1
        maxChange = -np.inf
        for s in range(nS):
            a = policy[s] # We have a deterministic policy, no need to iterate over actions in this case
            
            # Let's check the next moves we get from starting in state s and applying action a
            new_value_func = 0.0
            for nextMove in P[s][a]:
                probability, nextstate, reward, terminal = nextMove
                new_value_func += probability * (reward + gamma * value_function[nextstate]) # if policy wouldn't be deterministic  we would have to multiply all this with probability given by each a, pi(a|s)
                
            maxChange = max(maxChange, abs(new_value_func - value_function[s]))
            value_function[s] = new_value_func
            
    print(f"Policy evaluation converged after {numIters} iterations")
        
	############################

    return value_function

def policy_improvement(P, nS, nA, value_from_policy, policy, gamma=0.9):
    """Given the value function from policy improve the policy.
    
    Parameters
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    value_from_policy: np.ndarray
        The value calculated from the policy
    policy: np.array
        The previous policy.
    
    Returns
    -------
    new_policy: np.ndarray[nS]
        An array of integers. Each integer is the optimal action to take
        in that state according to the environment dynamics and the
        given value function.
    """

    new_policy = np.zeros(nS, dtype='int') # Default is left action

	############################
	# YOUR IMPLEMENTATION HERE #
    
    # Go through each state
    for s in range(nS):
        # Evaluate each actions
        value_per_action = np.zeros(shape=(nA,))
        for a in range(nA):
            action_val = 0.0
            # Get the model dynamics for this action and compute this action value
            for nextMove in P[s][a]:
                probability, nextstate, reward, terminal = nextMove
                action_val += probability * (reward + gamma * value_from_policy[nextstate]) # Here we take one step ahead then use the estimated value function of the next step
            
            value_per_action[a] = action_val
        
        # Choose the best action in the given state 
        best_action = np.argmax(value_per_action)
        new_policy[s] = best_action

    ############################
    return new_policy


def policy_iteration(P, nS, nA, gamma=0.9, tol=10e-3):
    """Runs policy iteration.
        You should call the policy_evaluation() and policy_improvement() methods to
        implement this method.
        
        Parameters
        ----------
        P, nS, nA, gamma:
            defined at beginning of file
        tol: float
            tol parameter used in policy_evaluation()
        Returns:
        ----------
        value_function: np.ndarray[nS]
        policy: np.ndarray[nS]
    """

    value_function = np.zeros(nS)
    policy = np.zeros(nS, dtype=int)

    ############################
    # YOUR IMPLEMENTATION HERE #
    print("Starting to improve policies")
    numIters = 0
    while True:
        numIters += 1
        print(f"PI step {numIters}: ")
        value_function = policy_evaluation(P, nS, nA, policy, gamma, tol)
        new_policy = policy_improvement(P, nS, nA, value_from_policy=value_function, policy=policy, gamma=gamma)
        isPolicyStable = not np.any(new_policy != policy) 
        
        if isPolicyStable:
            break
            
        policy = new_policy
    
    print(f"Got a stable policy after {numIters} iterations!")
    
    ############################
    return value_function, policy


gamma = 0.9
best_V, best_PI = policy_iteration(env.P, env.nS, env.nA, gamma=gamma, tol=10e-3)


Starting to improve policies
PI step 1: 
Policy evaluation converged after 1 iterations
PI step 2: 
Policy evaluation converged after 2 iterations
PI step 3: 
Policy evaluation converged after 3 iterations
PI step 4: 
Policy evaluation converged after 4 iterations
PI step 5: 
Policy evaluation converged after 5 iterations
PI step 6: 
Policy evaluation converged after 6 iterations
PI step 7: 
Policy evaluation converged after 7 iterations
Got a stable policy after 7 iterations!


In [6]:
# Now let's run the episode with this policy found !
runEpisode(env, policy=best_PI, maxSteps=1000)

Agent was in state 0, took action 1, now in state 4
Agent was in state 4, took action 1, now in state 8
Agent was in state 8, took action 2, now in state 9
Agent was in state 9, took action 1, now in state 13
Agent was in state 13, took action 2, now in state 14
Agent was in state 14, took action 2, now in state 15
Episode reward: 1.0


In [7]:
# Now let's implement value iteration algorithm, which in general can converge faster !
def value_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
    """
    Learn value function and policy by using value iteration method for a given
    gamma and environment.
    
    Parameters:
    ----------
    P, nS, nA, gamma:
        defined at beginning of file
    tol: float
        Terminate value iteration when
            max |value_function(s) - prev_value_function(s)| < tol
    Returns:
    ----------
    value_function: np.ndarray[nS]
    policy: np.ndarray[nS]
    """

    value_function = np.zeros(nS)
    policy = np.zeros(nS, dtype=int)
	############################
    # YOUR IMPLEMENTATION HERE #
    
    # Step 1: Value iteration in place. Its output is the optimal policy according to the Bellman equation
    maxChange = np.inf
    numIters = 0
    while maxChange > tol:
        numIters += 1
        maxChange = 0.0
        for s in range(nS):
            # Update the value of s by moving towards the action that maximizes the value according to the model dynamics
            bestActionValue = -np.inf
            for a in range(nA):
                value_for_thisAction = 0.0
                
                # Check model dynamics from going from state s and action a
                for nextMove in P[s][a]:
                    probability, nextstate, reward, terminal = nextMove
                    value_for_thisAction += probability * (reward + gamma * value_function[nextstate])
                    
                if value_for_thisAction > bestActionValue:
                    bestActionValue = value_for_thisAction
                
            maxChange = max(maxChange, abs(value_function[s] - bestActionValue))
            value_function[s] = bestActionValue
            
    print(f"Value iteration converged after {numIters}")
    
    # Now extract the best policy from the value computed above
    policy = policy_improvement(P, nS, nA, value_function, policy, gamma)

    ############################
    return value_function, policy

gamma = 0.9
best_value, best_policy = value_iteration(env.P, env.nS, env.nA, gamma=gamma, tol=10e-3)


Value iteration converged after 7


### Now let's run the episode with this policy found by value iteration
### Note that you may need to increase max number of Steps or run it several times if you select is_slippery !
### Try to play with map names and stochastic parameter is_slippery than run again all above cells including this

In [8]:
runEpisode(env, policy=best_PI, maxSteps=1000)

Agent was in state 0, took action 1, now in state 4
Agent was in state 4, took action 1, now in state 8
Agent was in state 8, took action 2, now in state 9
Agent was in state 9, took action 1, now in state 13
Agent was in state 13, took action 2, now in state 14
Agent was in state 14, took action 2, now in state 15
Episode reward: 1.0
